# Configure Code to run on GPU
* Do _not_ use GPU 0; other experiements may accidentally take over

In [1]:
import tensorflow as tf
import os
tf.debugging.set_log_device_placement(False) # displays which devices operations are assigned to

In [2]:
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))
# tf.config.experimental.list_physical_devices('GPU')

Num GPUs Available: 8


In [3]:
from functions.tf_config import config_gpu
gpu_index = 5               # change this value if desired gpu is occupied
config_gpu(gpu_index, True) # sets desired GPU as the only visible GPU for operations

8 Physical GPUs, 1 Logical GPU
Configured to run on GPU 5


In [4]:
try:
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    c = tf.matmul(a, b)

except RuntimeError as e:
    print(e)
    
print(c)
print("\nPID: ", os.getpid())
# for some reason, log shows GPU 0 but nvidia-smi shows the correct GPU
# suspect that log is displaying a virtual/ logical GPU running _on_ correct GPU

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)

PID:  51520


# Run MNIST Experiment on GPU

In [5]:
from tensorflow import keras
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, xtest = x_train / 255.0, x_test / 255.0

In [6]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
])

In [7]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.6631677 ,  0.5175778 , -0.34513354,  0.5882275 ,  0.46976972,
         0.07277152,  0.34676564, -0.10050498,  0.5259352 ,  0.4911097 ]],
      dtype=float32)

In [8]:
tf.nn.softmax(predictions).numpy()

array([[0.13433951, 0.11613814, 0.04901218, 0.12464005, 0.11071642,
        0.07443856, 0.0979021 , 0.06259575, 0.11711281, 0.1131045 ]],
      dtype=float32)

In [9]:
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn(y_train[:1], predictions).numpy()

2.5977812

In [10]:
model.compile(optimizer='adam',
             loss=loss_fn,
             metrics=['accuracy'])

In [11]:
model.fit(x_train,y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2981 - accuracy: 0.9143
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1422 - accuracy: 0.9577
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1088 - accuracy: 0.9668
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0894 - accuracy: 0.9730
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0766 - accuracy: 0.9766


In [12]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 12.0038 - accuracy: 0.9782


[12.00384521484375, 0.9782000184059143]

In [13]:
probability_model = keras.Sequential([
    model,
    keras.layers.Softmax()
])

In [14]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)>